# Function to determine the shortest path to fulfill an order

## LP model to determine the shortest path
To determine the shortest path to fulfill an order, we can implement a linear programming model which takes the order items that need to be obtained, and determines the shortest path to collect them all. The model assumes starting from the packaging area, collecting all items, and returning to the packaging area. The model is described below. 

#### Sets, parameters, variables
$I$ : The set of items in the order, as well as the packaging section (labelled as $0$). Note, we can call the order items the nodes to hit.

$d_{i j} :$ The distances between each node $i, j \in I$.

$n :$ The number of items in the order $(I/\{0\})$.

$x_{i j} :$ The decision variable, it equals $1$ if you travel from node $i$ to node $j$ and $0$ if not. 

$y_{i} :$ A dummy decision variable which tracks the number of nodes visited before node $i$, this is used to ensure we do not revisit the packaging station prior to collecting all the items.

#### Model
Min $$\sum_{i \in I} \sum_{j \in J} x_{i j} d_{i j}$$

Subject to $$\sum_{i \in I} x_{i j} = 1 \quad \forall j \in I \qquad (1)$$
$$\sum_{j \in I} x_{i j} = 1 \quad \forall i \in I \qquad (2)$$
$$x_{i i} = 0 \quad \forall i \in I \qquad (3)$$
$$x_{i j} + x_{j i} \leq 1 \quad \forall i \in I, j \in I \quad (4)$$
$$u_i - u_j + nx_{ij} \leq n - 1 \quad \forall i \neq j \in I/\{0\} \quad (5)$$

$$x_{i j} \in \{0, 1\}$$
$$y_{i j} \in \mathbb{Z}^+ $$


Note - constraint $(1)$ ensures that there is one arc into every node, and constraint $(2)$ ensures that there is one arc out of every node. Constraint $(3)$ ensures that you don't go from node i to node i. Constraint $(4)$ ensures that you cannot return back to the node you just came from. Constraint $(5)$ ensures that you don't return to packaging until all nodes have been hit. 

The final constraint - constraint $(5)$ was taken from an mathematical article entitled "Integer Programming Formulation of Traveling Salesman Problems", which discusses the formulation of Travelling Salesman Problem when it is required that there are no sub-networks. Reference - Miller, C, A Tucker, and R Zemlin. “Integer Programming Formulation of Traveling Salesman Problems.” Journal of the ACM 7.4 (1960): 326–329. Web.

Reference link - https://dl-acm-org.eux.idm.oclc.org/doi/pdf/10.1145/321043.321046 (got it from online library).


## Implementation of shortest path function
We can define this model as a function which takes in the order, and returns the shortest path. This has been implemented below. Note that we firstly load in all the distances, and define the sets, parameters, and variables before adding constraints, the objective, and producing the output.

In [1]:
# Import packages
import numpy as np
import xpress as xp
import pandas as pd

In [2]:
def shortest_path(order):
    '''
    A function which takes as input an order which is a list of item numbers. 
    
    Returns the shortest path to collect all items, when starting and ending at the packaging station, and the 
    total distance to fulfill the order. 
    '''
    
    # SORTING SETS, PARAMETERS, VARIABLES
    
    # Sort the order into ascending order
    order.sort()
    
    # Drop the 0 values
    order = [i for i in order if i != 0]
    
    # Obtain the length of n
    n = len(order)
    
    # Create a copy to generate the I set
    order_2 = order.copy()
    
    # Define I to be the order with 0 attached 
    order_2.insert(0, 0)
    I = order_2
    
    # Insert "Packaging into the order list so we can obtain the desired columns 
    order.insert(0,"Packaging")
    
    # Import data file of distances
    distances_data = pd.read_excel('DistanceMatrix.xlsx', sheet_name = "DistanceMatrix Meters")

    # Drop the first column of indices
    d_dat = distances_data.drop(columns = "Index")
    
    # Select desired rows and columns
    d_data = distances_data.loc[distances_data["Index"].isin(order)] # Filter rows
    d_data = d_data[order] # Filter columns

    # Rename the packaging column to 0
    d_data = d_data.rename(columns = {"Packaging" : "0"}) 

    # Select and drop the final row
    first_r = d_data.loc[d_data["0"] == 0]
    d_data = d_data.drop(index = [96])

    # Concat the dataframes so final row is first row
    d = pd.concat([first_r, d_data])
    
    # Turn the data into an array for use
    d = d.to_numpy()
    
    # Generate a list of indices for the set
    I_ind = list(range(0, len(I)))    
    
    # Define the x variable
    x = np.array([xp.var(vartype = xp.binary, name = 'x_{0}_{1}'.format(i, j)) for i in I for j in I], 
                 dtype = xp.npvar).reshape(len(I), len(I))
    
    # Define the y variable
    y = np.array([xp.var(vartype = xp.integer, name = 'y_{0}'.format(i)) for i in I], 
                 dtype = xp.npvar)
    
    
    
    # DEFINE THE PROBLEM, DECLARE VARIABLES AND CONSTRAINTS
    
    # Set the problem
    prob = xp.problem(name = "Prob")
    
    # Add the decision variable to the problem
    prob.addVariable(x)
    prob.addVariable(y)

    
    # Add the constraints:
    
    # only one arc into each node
    prob.addConstraint(
        xp.Sum(x[i, j] for i in I_ind) == 1 for j in I_ind
    )

    # only one arc out of each node
    prob.addConstraint(
        xp.Sum(x[i, j] for j in I_ind) == 1 for i in I_ind
    )

    # have to go to a different node
    prob.addConstraint(
        x[i, i] == 0 for i in I_ind
    )

    # can't go back to the node you just came from, unless there is only 1 item to collect
    if len(I_ind) != 2:
        prob.addConstraint(
            x[i, j] + x[j, i] <= 1 for i in I_ind for j in I_ind
        )
        
    # no sub-networks
    for i in I_ind :
        for j in I_ind :
            if i != j and i != 0 and j != 0:
                prob.addConstraint(
                    y[i] - y[j] + n*x[i, j] <= n - 1
                )
        
        
    # DEFINE AND ADD OBJECTIVE
    
    # Define the objective function
    obj = xp.Sum(xp.Sum(x[i, j]*d[i, j] for i in I_ind) for j in I_ind)

    # Set the problems objective function
    prob.setObjective(obj, sense = xp.minimize)
    
    
    # WRITE AND SOLVE PROBLEM
    # Write and solve the problem
    prob.write("problem","lp") # Used to look for cause of infeasibility
    prob.solve()
    
    
    # DEFINE OUTPUTS
    
    # Obtain optimal x values, and objective value
    soln = prob.getSolution(x)
    total_distance = prob.getObjVal()
    
    # Set an empty array for arcs
    arcs = []
    
    # Determine the arcs
    for i in I_ind :
        for j in I_ind :
            if soln[i, j] == 1 :
                arcs.append([I[i], I[j]])

    # Obtain the y solution
    y_soln = prob.getSolution(y)
    
    # Create a dataframe of the nodes and the y values
    df = pd.DataFrame({"I" : I, 'y' : y_soln})
    
    # Sort based on the y values
    df2 = df.sort_values(by = ["y"])

    # The order of collection
    sequence = df2['I'].values.tolist()
    
    return total_distance, sequence

## Test Functions
To ensure the function is running correctly, define $5$ test orders and check output. 

In [3]:
# Test 1
total_distance, sequence = shortest_path([6, 0, 0, 0, 0])
print(f"The total distance is {total_distance}m")
print(f"The sequence is {sequence}")

Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
FICO Xpress v9.2.2, Community, solve started 21:10:44, Mar 13, 2024
Heap usage: 390KB (peak 422KB, 120KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
         6 rows            6 cols           10 elements         6 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 394KB (peak 422KB, 120KB system)
Will try to keep branch and bound tree memory usage below 7.5GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  42.000000   .0000000
------- optimal --

In [4]:
# Test 2
total_distance, sequence = shortest_path([50, 30, 0, 0, 0])
print(f"The total distance is {total_distance}m")
print(f"The sequence is {sequence}")

FICO Xpress v9.2.2, Community, solve started 21:10:44, Mar 13, 2024
Heap usage: 396KB (peak 428KB, 123KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        20 rows           12 cols           42 elements        12 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
Presolve finished in 0 seconds
Heap usage: 401KB (peak 428KB, 123KB system)
Will try to keep branch and bound tree memory usage below 7.5GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  66.000000   .0000000
------- optimal --------
Concurrent statistics:
      Dual: 0 simplex iterations, 0.00s
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0         66.000000      D      0     0        .000000     0
Dual solved problem
  0 simplex iterations in 0.0

In [5]:
# Test 3
total_distance, sequence = shortest_path([49, 18, 76, 0, 0])
print(f"The total distance is {total_distance}m")
print(f"The sequence is {sequence}")

FICO Xpress v9.2.2, Community, solve started 21:10:44, Mar 13, 2024
Heap usage: 400KB (peak 432KB, 126KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        34 rows           20 cols           82 elements        20 entities
Presolved problem has:
        20 rows           15 cols           54 elements        15 entities
Presolve finished in 0 seconds
Heap usage: 435KB (peak 435KB, 126KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  3.00e+00] / [ 5.00e-01,  1.50e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 9.00e+00,  7.50e+01] / [ 9.00e+00,  7.50e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 7.5GB
 *** Solution found:   150.000000   Time:   0.00    Heuristic: e ***
Starting concurrent solve

In [6]:
# Test 4
total_distance, sequence = shortest_path([88, 70, 35, 2, 0])
print(f"The total distance is {total_distance}m")
print(f"The sequence is {sequence}")

FICO Xpress v9.2.2, Community, solve started 21:10:44, Mar 13, 2024
Heap usage: 418KB (peak 451KB, 128KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        52 rows           30 cols          136 elements        30 entities
Presolved problem has:
        32 rows           24 cols           96 elements        24 entities
Presolve finished in 0 seconds
Heap usage: 455KB (peak 480KB, 128KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  4.00e+00] / [ 2.50e-01,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  3.00e+00] / [ 7.50e-01,  1.00e+00]
  Objective      [min,max] : [ 9.00e+00,  8.10e+01] / [ 9.00e+00,  8.10e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 7.5GB
 *** Solution found:   186.000000   Time:   0.00    Heuristic: e ***
 *** Solution found:   16

In [7]:
# Test 5
total_distance, sequence = shortest_path([39, 18, 29, 83, 49])
print(f"The total distance is {total_distance}m")
print(f"The sequence is {sequence}")

FICO Xpress v9.2.2, Community, solve started 21:10:44, Mar 13, 2024
Heap usage: 427KB (peak 459KB, 131KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        74 rows           42 cols          204 elements        42 entities
Presolved problem has:
        47 rows           35 cols          150 elements        35 entities
Presolve finished in 0 seconds
Heap usage: 465KB (peak 490KB, 131KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  5.00e+00] / [ 2.50e-01,  1.25e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 9.00e+00,  8.10e+01] / [ 9.00e+00,  8.10e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 7.5GB
 *** Solution found:   264.000000   Time:   0.01    Heuristic: e ***
Starting concurrent solve

## Function to determine the total distance
We can now create a function `distance()` which takes a list of orders, calls `shortest_path()` and returns the minimum distance required to process all orders. 

This has been implemented below, by firstly loading in the order data, and then defining and running the function. 

In [8]:
# Load in the data file of orders
orders = pd.read_excel('OrderList.xlsx', sheet_name = "Orders")

# Drop the order numbers
orders = orders.drop(columns = "Order No.")

# Display a snippet of the data frame
orders

,Position 1,Position 2,Position 3,Position 4,Position 5
0,50,30,0,0,0
1,49,18,76,0,0
2,72,52,51,41,35
3,50,4,0,0,0
4,76,19,26,80,6
...,...,...,...,...,...
1995,60,46,35,0,0
1996,8,43,70,77,31
1997,46,0,0,0,0
1998,90,23,64,35,0


In [9]:
def distance(orders) :
    '''
    A function which takes as input a data frame of orders. 
    
    Returns the total distance required to fulfill all orders.
    
    '''
    
    # Define the intital total distance as 0
    tot_distance = 0

    # Loop over the number of orders
    for i in range(0, len(orders)):
        
        # Obtain the order information from the data frame
        order = orders.iloc[i].values.tolist()
        
        # Run the shortest path function and obtain the distance required for given order
        total_distance, sequence = shortest_path(order)
        
        # Add this distance to the running total
        tot_distance += total_distance
    
    # Return the total distance
    return tot_distance

In [10]:
# Run the function for the above order list (only first 20 for computation time)
tot_distance = distance(orders.head(20))

# Print outcome
print(f"The total distance is {tot_distance}m")

FICO Xpress v9.2.2, Community, solve started 21:10:45, Mar 13, 2024
Heap usage: 396KB (peak 428KB, 134KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        20 rows           12 cols           42 elements        12 entities
Presolved problem has:
         0 rows            0 cols            0 elements         0 entities
Presolve finished in 0 seconds
Heap usage: 401KB (peak 428KB, 134KB system)
Will try to keep branch and bound tree memory usage below 7.5GB
Starting concurrent solve with dual (1 thread)

 Concurrent-Solve,   0s
            Dual        
    objective   dual inf
 D  66.000000   .0000000
------- optimal --------
Concurrent statistics:
      Dual: 0 simplex iterations, 0.00s
Optimal solution found
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0         66.000000      D      0     0        .000000     0
Dual solved problem
  0 simplex iterations in 0.0


Starting root cutting & heuristics
Deterministic mode with up to 1 additional thread
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
*           54.000000    54.000000      1                  0.00%       0      0
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 5.400000000000000e+01
Final MIP bound                       : 5.400000000000000e+01
  Solution time / primaldual integral :      0.01s/ 76.814063%
  Number of solutions found / nodes   :         1 /         1
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max integer violation     (abs    ) :       0.0
FICO Xpress v9.2.2, Community, solve started 21:10:46, Mar 13, 2024
Heap usage: 427KB (peak 459KB, 143KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        74 rows           42 cols          204 elements        42 entities
Presolved problem has:
  

  10  K    252.000000   156.000000      2      1      2   38.10%       8      0
  11  K    252.000000   156.000000      2      0      1   38.10%       7      0
  12  G    252.000000   156.000000      2      7      0   38.10%       7      0
  13  G    252.000000   156.000000      2      6     13   38.10%       7      0
Heuristic search 'R' started
Heuristic search 'R' stopped
 
Cuts in the matrix         : 6
Cut elements in the matrix : 75

Starting tree search.
Deterministic mode with up to 4 running threads and up to 8 tasks.
Heap usage: 3431KB (peak 5920KB, 1071KB system)
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
       1   252.000000   163.200000      2      2      1   35.24%       7      0
a      2   168.000000   163.200000      3      2      3    2.86%       0      0
       4   168.000000   163.200000      3      2      3    2.86%       7      0
       7   168.000000   163.200000      3      1      3    2.86%      13      0
 *** Search compl

q          114.000000   114.000000      5                 -0.00%       0      0
 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.140000000000000e+02
Final MIP bound                       : 1.140000000000000e+02
  Solution time / primaldual integral :      0.03s/ 49.596889%
  Number of solutions found / nodes   :         5 /         1
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max integer violation     (abs    ) :       0.0
FICO Xpress v9.2.2, Community, solve started 21:10:47, Mar 13, 2024
Heap usage: 427KB (peak 459KB, 157KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        74 rows           42 cols          204 elements        42 entities
Presolved problem has:
        47 rows           35 cols          150 elements        35 entities
Presolve finished in 0 seconds
Heap usage: 465KB (peak 490KB, 157KB system)

Coefficient range 

FICO Xpress v9.2.2, Community, solve started 21:10:47, Mar 13, 2024
Heap usage: 427KB (peak 459KB, 164KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        74 rows           42 cols          204 elements        42 entities
Presolved problem has:
        47 rows           35 cols          150 elements        35 entities
Presolve finished in 0 seconds
Heap usage: 465KB (peak 490KB, 164KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  5.00e+00] / [ 2.50e-01,  1.25e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 3.00e+00,  7.50e+01] / [ 3.00e+00,  7.50e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 7.5GB
 *** Solution found:   222.000000   Time:   0.00    Heuristic: e ***
Starting concurrent solve

 *** Search completed ***
Uncrunching matrix
Final MIP objective                   : 1.680000000000000e+02
Final MIP bound                       : 1.680000000000000e+02
  Solution time / primaldual integral :      0.01s/ 61.241292%
  Number of solutions found / nodes   :         1 /         1
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max integer violation     (abs    ) :       0.0
FICO Xpress v9.2.2, Community, solve started 21:10:48, Mar 13, 2024
Heap usage: 427KB (peak 459KB, 177KB system)
Minimizing MILP Prob using up to 4 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        74 rows           42 cols          204 elements        42 entities
Presolved problem has:
        47 rows           35 cols          150 elements        35 entities
Presolve finished in 0 seconds
Heap usage: 465KB (peak 490KB, 177KB system)

Coefficient range                    original                 solved        
  Coefficients   [min

Heuristic search 'R' started
Heuristic search 'R' stopped
 
Cuts in the matrix         : 7
Cut elements in the matrix : 104

Starting tree search.
Deterministic mode with up to 4 running threads and up to 8 tasks.
Heap usage: 3436KB (peak 5922KB, 1102KB system)
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
       1   198.000000   129.600000      2      2      1   34.55%       6      0
       2   198.000000   129.600000      2      2      3   34.55%       8      0
       3   198.000000   129.600000      2      2      3   34.55%       4      0
a      3   174.000000   129.600000      3      2      3   25.52%       0      0
       4   174.000000   129.600000      3      2      3   25.52%       1      0
       5   174.000000   129.600000      3      1      1   25.52%       2      0
       6   174.000000   129.600000      3      1      4   25.52%       4      0
       7   174.000000   129.600000      3      1      4   25.52%       1      0
b      7   168.0